In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import io
import warnings
warnings.filterwarnings('ignore')
import ahpy

# Carga de los archivos
shipping = pd.read_excel("Datos_limpios.xlsx", sheet_name="Shipping Detail Report")
labor = pd.read_excel("Datos_limpios.xlsx", sheet_name="Labor Activity Report")

# Unificar formatos de SKU, fechas, y tipos de datos
shipping["SKU"] = shipping["SKU"].astype(str).str.upper()
labor["SKU"] = labor["SKU"].astype(str).str.upper()

# dividimos por fecha
#
#
#


shipping_summary = (
    shipping.groupby("SKU")
    .agg({
        "Qty Shipped": "sum",
        "Weight [Kg]": "sum",
        "Boxes": "sum"
    })
    .reset_index()
)

# clasificacion ABC pareto
shipping_summary = shipping_summary.sort_values("Qty Shipped", ascending=False)
shipping_summary["cum%"] = 100 * shipping_summary["Qty Shipped"].cumsum() / shipping_summary["Qty Shipped"].sum()

shipping_summary["ABC_class"] = pd.cut(
    shipping_summary["cum%"],
    bins=[0, 80, 90, 100],
    labels=["A", "B", "C"]
)

comparisons = {
    ('Qty Shipped', 'Weight [Kg]'): 5, ('Qty Shipped', 'Boxes'):9,
    ('Weight [Kg]', 'Boxes'): 4
}

criteria = ahpy.Compare('Criterios', comparisons, precision=3, random_index='saaty')

print(criteria.report())
print("Pesos:", criteria.target_weights)
print("Consistencia:", criteria.consistency_ratio)

# ABC con AHP
# Normalizar columnas y crear nuevas con sufijo "_norm"
for col in ["Qty Shipped", "Weight [Kg]", "Boxes"]:
    shipping_summary[f"{col}_norm"] = shipping_summary[col] / shipping_summary[col].max()

# Calcular AHP_score usando las columnas normalizadas
shipping_summary['AHP_score'] = (
    shipping_summary['Qty Shipped_norm'] * criteria.target_weights['Qty Shipped'] +
    shipping_summary['Weight [Kg]_norm'] * criteria.target_weights['Weight [Kg]'] +
    shipping_summary['Boxes_norm'] * criteria.target_weights['Boxes']
)

shipping_summary = shipping_summary.sort_values('AHP_score', ascending=False)
shipping_summary['cum_AHP%'] = shipping_summary['AHP_score'].cumsum() / shipping_summary['AHP_score'].sum()

def categoria(x):
    if x <= 0.8: return 'A'
    elif x <= 0.9: return 'B'
    else: return 'C'

shipping_summary['AHP_class'] = shipping_summary['cum_AHP%'].apply(categoria)

print(shipping_summary['AHP_class'].value_counts())
print(shipping_summary['ABC_class'].value_counts())

# metricas por clase A, puede cambiar
# miramos las clases en clase A en AHP cuanto porcentage representan de qty, weight y boxes
# Filtramos solo los de clase A
df_A = shipping_summary[shipping_summary['AHP_class'] == 'A']

# Calculamos los totales
total_qty = shipping_summary['Qty Shipped'].sum()
total_weight = shipping_summary['Weight [Kg]'].sum()
total_boxes = shipping_summary['Boxes'].sum()

# Calculamos los totales de la clase A
A_qty = df_A['Qty Shipped'].sum()
A_weight = df_A['Weight [Kg]'].sum()
A_boxes = df_A['Boxes'].sum()

# Calculamos los porcentajes
pct_qty_A = 100 * A_qty / total_qty
pct_weight_A = 100 * A_weight / total_weight
pct_boxes_A = 100 * A_boxes / total_boxes

# Mostramos resultados
print(f"Clase A representa:")
print(f"- {pct_qty_A:.2f}% del total de Qty Shipped")
print(f"- {pct_weight_A:.2f}% del total de Weight [Kg]")
print(f"- {pct_boxes_A:.2f}% del total de Boxes")


# boxplot para gráficar
# Crear figura y ejes
fig, axes = plt.subplots(3, 2, figsize=(14, 8))  # 3 filas, 2 columnas

# Primera fila
sns.boxplot(data=shipping_summary, hue='AHP_class', y='Qty Shipped', ax=axes[0, 0])
axes[0, 0].set_title('Qty Shipped - AHP_class')

sns.boxplot(data=shipping_summary, hue='ABC_class', y='Qty Shipped', ax=axes[0, 1])
axes[0, 1].set_title('Qty Shipped - ABC_class')

# Segunda fila
sns.boxplot(data=shipping_summary, hue='AHP_class', y='Weight [Kg]', ax=axes[1, 0])
axes[1, 0].set_title('Weight [Kg] - AHP_class')

sns.boxplot(data=shipping_summary, hue='ABC_class', y='Weight [Kg]', ax=axes[1, 1])
axes[1, 1].set_title('Weight [Kg] - ABC_class')

# Tercera fila
sns.boxplot(data=shipping_summary, hue='AHP_class', y='Boxes', ax=axes[2, 0])
axes[2, 0].set_title('Boxes - AHP_class')

sns.boxplot(data=shipping_summary, hue='ABC_class', y='Boxes', ax=axes[2, 1])
axes[2, 1].set_title('Boxes - ABC_class')

# Ajustar diseño
plt.tight_layout()
plt.show()

# métricas de similitud
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.metrics.pairwise import cosine_similarity

features = ['Qty Shipped', 'Weight [Kg]', 'Boxes']
X = shipping_summary[features].values
scaler = StandardScaler()
Xs = scaler.fit_transform(X)
df_scaled = shipping_summary.copy()
df_scaled[features] = Xs

# función para métricas por clase
def metrics_por_clase(df, features):
    results = {}
    for clase, sub in df.groupby('AHP_class'):
        Xc = sub[features].values
        n = len(Xc)
        if n <= 1:
            results[clase] = {
                'n': n,
                'mean_cosine_similarity': np.nan,
                'median_cosine_similarity': np.nan,
                'mean_euclid_distance': np.nan,
                'score_cv': (sub['AHP_score'].std()/sub['AHP_score'].mean()) if sub['AHP_score'].mean()!=0 else np.nan
            }
            continue

        # similitud coseno par-a-par (sin diagonal)
        cos = cosine_similarity(Xc)
        # extraer valores por encima de diagonal
        iu = np.triu_indices_from(cos, k=1)
        cos_values = cos[iu]

        # distancias euclidianas
        dists = pairwise_distances(Xc, metric='euclidean')
        dists_values = dists[iu]

        results[clase] = {
            'n': n,
            'mean_cosine_similarity': float(np.mean(cos_values)),
            'median_cosine_similarity': float(np.median(cos_values)),
            'mean_euclid_distance': float(np.mean(dists_values)),
            'score_cv': float(sub['AHP_score'].std()/sub['AHP_score'].mean()) if sub['AHP_score'].mean()!=0 else np.nan
        }
    return pd.DataFrame(results).T

res_clase = metrics_por_clase(df_scaled, features)
res_clase.head()

# silhouette requiere al menos 2 labels y más de 1 muestra por label en la práctica
labels = df_scaled['AHP_class'].values
Xall = df_scaled[features].values

sil = silhouette_score(Xall, labels)  # entre -1 y 1
db = davies_bouldin_score(Xall, labels)
ch = calinski_harabasz_score(Xall, labels)

print("Silhouette:", sil)
print("Davies-Bouldin:", db)
print("Calinski-Harabasz:", ch)







In [ ]:
# app.py
"""
App Streamlit: clasificación ABC vs AHP para Shipping y Labor.
Mantener nombres de pestañas: "Shipping Detail Report" y "Labor Activity Report".
"""

import streamlit as st
import pandas as pd
import numpy as np
import io
import base64
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import ahpy

st.set_page_config(layout="wide", page_title="ABC vs AHP Dashboard")

# ---------------------------
# Util: funciones centrales
# ---------------------------

def compute_summary(df, sku_col='SKU', aggregations=None):
    """
    Agrupa por SKU y devuelve un resumen con Qty Shipped, Weight [Kg], Boxes.
    Si las columnas no existen, intenta encontrar nombres parecidos.
    """
    # Por defecto, usar estas columnas si existen
    defaults = {'Qty Shipped': 'Qty Shipped', 'Weight [Kg]': 'Weight [Kg]', 'Boxes': 'Boxes'}
    if aggregations is None:
        aggregations = defaults
    # Asegurar existencia de columnas: si no existen, crear columnas con 0
    for col in aggregations.values():
        if col not in df.columns:
            df[col] = 0
    # Agrupar
    summary = df.groupby(sku_col).agg({
        aggregations['Qty Shipped']: 'sum',
        aggregations['Weight [Kg]']: 'sum',
        aggregations['Boxes']: 'sum'
    }).reset_index().rename(columns={
        aggregations['Qty Shipped']: 'Qty Shipped',
        aggregations['Weight [Kg]']: 'Weight [Kg]',
        aggregations['Boxes']: 'Boxes'
    })
    # Evitar NaNs
    summary[['Qty Shipped','Weight [Kg]','Boxes']] = summary[['Qty Shipped','Weight [Kg]','Boxes']].fillna(0)
    return summary

def compute_abc(summary, qty_col='Qty Shipped', cuts=[80,90]):
    """
    Clasificación ABC clásica por Pareto basada en `qty_col`.
    - cuts: lista con percentiles acumulativos (ej [80,90]) => A: <=80, B: >80-90, C: >90-100
    Devuelve summary con 'cum%', 'ABC_class'
    """
    df = summary.copy()
    # Orden descendente por qty
    df = df.sort_values(qty_col, ascending=False).reset_index(drop=True)
    total = df[qty_col].sum()
    # Si total 0, evitar división por cero; asignar 0%
    if total == 0:
        df['cum%'] = 0.0
    else:
        df['cum%'] = (df[qty_col].cumsum() / total) * 100
    # Definir cortes
    cuts_sorted = sorted(cuts)
    def classify(x):
        if x <= cuts_sorted[0]:
            return 'A'
        elif x <= cuts_sorted[1]:
            return 'B'
        else:
            return 'C'
    df['ABC_class'] = df['cum%'].apply(classify)
    return df

def compute_ahp(summary, features, comparisons_dict, cuts=[80,90]):
    """
    Ejecuta AHP con ahpy usando comparisons_dict (pares de comparaciones) y calcula AHP_score.
    - features: lista de columnas sobre las que construir el score (columnas numéricas de summary)
    - comparisons_dict: diccionario con comparaciones en el formato {'A': {'B': value, 'C': value}, 'B': {'C':value}}
      (compatible con ahpy.Compare)
    Devuelve summary con columnas normalizadas, 'AHP_score', 'cum_AHP%', 'AHP_class' y retorna también el objeto criteria de ahpy.
    """
    df = summary.copy().reset_index(drop=True)
    # Normalizar columnas (col / col.max())
    for col in features:
        if col not in df.columns:
            df[col] = 0.0
        maxv = df[col].max()
        if maxv == 0:
            df[f'{col}_norm'] = 0.0
        else:
            df[f'{col}_norm'] = df[col] / maxv

    # Ejecutar ahpy: Construir objeto Compare
    try:
        criteria = ahpy.Compare('criteria', comparisons=comparisons_dict, precision=6)
        weights = criteria.target_weights  # dict {feature: weight}
        # Asegurar que todos features tengan un peso (si el usuario escribió pesos directos, manejar)
        weights_list = [weights.get(f, 0) for f in features]
    except Exception as e:
        # Si ahpy falla, devolver pesos iguales
        st.warning("ahpy no pudo construir la matriz AHP con las comparaciones dadas. Se usarán pesos iguales.")
        weights_list = [1.0/len(features)]*len(features)
        criteria = None

    # Calcular AHP_score = suma(weights[i] * feature_norm)
    # Si criteria existe, usar sus pesos; si no, usar weights_list
    norm_cols = [f'{col}_norm' for col in features]
    w = np.array(weights_list, dtype=float)
    norms = df[norm_cols].fillna(0).values
    # Normalizar pesos para que sumen 1 si no están normalizados
    if w.sum() == 0:
        w = np.ones(len(features))/len(features)
    else:
        w = w / w.sum()
    df['AHP_score'] = norms.dot(w)
    # cumsum por puntuación descendente
    df = df.sort_values('AHP_score', ascending=False).reset_index(drop=True)
    total_ahp = df['AHP_score'].sum()
    if total_ahp == 0:
        df['cum_AHP%'] = 0.0
    else:
        df['cum_AHP%'] = (df['AHP_score'].cumsum() / total_ahp) * 100
    # Clasificación A/B/C con mismos cortes
    cuts_sorted = sorted(cuts)
    def classify_ahp(x):
        if x <= cuts_sorted[0]:
            return 'A'
        elif x <= cuts_sorted[1]:
            return 'B'
        else:
            return 'C'
    df['AHP_class'] = df['cum_AHP%'].apply(classify_ahp)

    return df, criteria

def compute_similarity_metrics(summary, features, class_col='AHP_class'):
    """
    Calcula métricas por clase:
    - mean & median cosine similarity (pairwise entre elementos dentro de cada clase)
    - mean euclidean distance (pairwise)
    - coeficiente de variación (std/mean) del AHP_score por clase
    También calcula índices globales: silhouette, Davies-Bouldin, Calinski-Harabasz (si aplicable).
    Devuelve: (df_metrics_por_clase, dict_global_indices)
    """
    df = summary.copy().reset_index(drop=True)
    results = []
    X = df[features].fillna(0).values
    labels = df[class_col].values
    # Por clase
    for cls in np.unique(labels):
        idx = np.where(labels == cls)[0]
        sub = X[idx]
        res = {'class': cls, 'n': len(idx)}
        if len(idx) < 2:
            res.update({
                'mean_cosine_sim': np.nan,
                'median_cosine_sim': np.nan,
                'mean_euclidean': np.nan,
                'cv_ahp_score': np.nan
            })
        else:
            # cosine similarity matriz (pairwise), tomar upper triangle sin diagonal
            cos_mat = cosine_similarity(sub)
            iu = np.triu_indices_from(cos_mat, k=1)
            cos_vals = cos_mat[iu]
            res['mean_cosine_sim'] = np.mean(cos_vals) if cos_vals.size>0 else np.nan
            res['median_cosine_sim'] = np.median(cos_vals) if cos_vals.size>0 else np.nan
            # euclidian distances
            dists = pairwise_distances(sub, metric='euclidean')
            dists_vals = dists[iu]
            res['mean_euclidean'] = np.mean(dists_vals) if dists_vals.size>0 else np.nan
            # Coef de variación del AHP_score en el grupo
            ahp_vals = df.loc[idx, 'AHP_score'].values
            if ahp_vals.mean() == 0:
                res['cv_ahp_score'] = np.nan
            else:
                res['cv_ahp_score'] = ahp_vals.std(ddof=0)/ahp_vals.mean()
        results.append(res)
    df_metrics = pd.DataFrame(results)

    # Global indices: necesitan al menos 2 clusters y cada cluster con >=2 elementos para silhouette
    global_idx = {}
    try:
        # silhouette requires >1 label and n_samples > n_labels
        if len(np.unique(labels)) > 1 and X.shape[0] > len(np.unique(labels)):
            # silhouette_score requiere al menos 2 elementos por cluster? Actually only total > n_clusters.
            global_idx['silhouette'] = silhouette_score(X, labels)
        else:
            global_idx['silhouette'] = np.nan
    except Exception:
        global_idx['silhouette'] = np.nan
    try:
        if len(np.unique(labels)) > 1:
            global_idx['davies_bouldin'] = davies_bouldin_score(X, labels)
            global_idx['calinski_harabasz'] = calinski_harabasz_score(X, labels)
        else:
            global_idx['davies_bouldin'] = np.nan
            global_idx['calinski_harabasz'] = np.nan
    except Exception:
        global_idx['davies_bouldin'] = np.nan
        global_idx['calinski_harabasz'] = np.nan

    return df_metrics, global_idx

# ---------------------------
# UI: Upload y selección inicial
# ---------------------------
st.title("Clasificación ABC vs AHP — Shipping & Labor")
st.markdown("**Instrucciones:** Subir un archivo Excel que contenga *exactamente* las hojas `Shipping Detail Report` y `Labor Activity Report` (no renombres las pestañas).")

uploaded = st.file_uploader("Sube archivo Excel (.xlsx) con ambas hojas", type=['xlsx'])

if not uploaded:
    st.info("Sube un archivo Excel para empezar. Asegúrate de que las hojas se llamen exactamente 'Shipping Detail Report' y 'Labor Activity Report'.")
    st.stop()

# Cargar archivos
try:
    xls = pd.ExcelFile(uploaded)
    available_sheets = xls.sheet_names
    if "Shipping Detail Report" not in available_sheets or "Labor Activity Report" not in available_sheets:
        st.error("El archivo no contiene una o ambas hojas requeridas: 'Shipping Detail Report' y 'Labor Activity Report'. Verifica el archivo.")
        st.stop()
    df_ship_raw = pd.read_excel(xls, sheet_name="Shipping Detail Report")
    df_labor_raw = pd.read_excel(xls, sheet_name="Labor Activity Report")
except Exception as e:
    st.exception(e)
    st.stop()

st.sidebar.header("Configuración general")
view_mode = st.sidebar.selectbox("Ver dataset", options=['Shipping','Labor'])
# Preview y selección de columna fecha
st.subheader("Preview y selección de columna fecha")

col1, col2 = st.columns(2)
with col1:
    st.write("**Shipping - head()**")
    st.dataframe(df_ship_raw.head())
    ship_date_col = st.selectbox("Selecciona columna fecha (Shipping)", options=list(df_ship_raw.columns), index=0, key='ship_date')
with col2:
    st.write("**Labor - head()**")
    st.dataframe(df_labor_raw.head())
    labor_date_col = st.selectbox("Selecciona columna fecha (Labor)", options=list(df_labor_raw.columns), index=0, key='labor_date')

# Convertir a datetime y pedir rango (aplicado a ambos)
df_ship_raw[ship_date_col] = pd.to_datetime(df_ship_raw[ship_date_col], errors='coerce')
df_labor_raw[labor_date_col] = pd.to_datetime(df_labor_raw[labor_date_col], errors='coerce')

min_date = min(df_ship_raw[ship_date_col].min(), df_labor_raw[labor_date_col].min())
max_date = max(df_ship_raw[ship_date_col].max(), df_labor_raw[labor_date_col].max())

st.sidebar.subheader("Filtrar rango de fechas (aplicado a ambos datasets)")
date_range = st.sidebar.date_input("Rango fechas", value=[min_date.date(), max_date.date()], min_value=min_date.date(), max_value=max_date.date())
start, end = pd.to_datetime(date_range[0]), pd.to_datetime(date_range[1])

# Aplicar filtro a ambos datasets
df_ship = df_ship_raw[(df_ship_raw[ship_date_col] >= start) & (df_ship_raw[ship_date_col] <= end)].copy()
df_labor = df_labor_raw[(df_labor_raw[labor_date_col] >= start) & (df_labor_raw[labor_date_col] <= end)].copy()

st.sidebar.write(f"Shipping filtrado: {df_ship.shape[0]} filas | Labor filtrado: {df_labor.shape[0]} filas")

# ---------------------------
# Parámetros ABC (umbral editable)
# ---------------------------
st.sidebar.subheader("Umbrales ABC (cortes acumulativos en %)")
default_cuts = [80, 90]
cut_a = st.sidebar.slider("A hasta (%)", min_value=1, max_value=99, value=default_cuts[0])
cut_b = st.sidebar.slider("B hasta (%)", min_value=cut_a+1, max_value=100, value=default_cuts[1])
cuts = [cut_a, cut_b]

# ---------------------------
# Selección columnas SKU y agregación
# ---------------------------
st.sidebar.subheader("Columnas clave y agregación")
sku_col_ship = st.sidebar.text_input("Nombre de columna SKU (Shipping)", value="SKU")
qty_col_ship = st.sidebar.text_input("Nombre Qty Shipped (Shipping)", value="Qty Shipped")
weight_col_ship = st.sidebar.text_input("Nombre Weight [Kg] (Shipping)", value="Weight [Kg]")
boxes_col_ship = st.sidebar.text_input("Nombre Boxes (Shipping)", value="Boxes")

sku_col_labor = st.sidebar.text_input("Nombre de columna SKU (Labor)", value="SKU")
# Sugerir métricas de labor (por ejemplo Hours, Headcount) - el usuario selecciona las columnas que quiera usar para AHP
# No forzamos nombres específicos para labor; permitimos seleccionar más abajo.

# ---------------------------
# Selección de variables para AHP (features)
# ---------------------------
st.sidebar.subheader("Selección de variables para AHP")
if view_mode == 'Shipping':
    # Mostrar columnas sugeridas para usar en AHP
    candidate_features_ship = [qty_col_ship, weight_col_ship, boxes_col_ship]
    use_features = st.sidebar.multiselect("Selecciona variables para AHP (Shipping)", options=candidate_features_ship, default=candidate_features_ship)
else:
    # Para labor, permitir seleccionar columnas del dataframe lab
    candidate_features_labor = list(df_labor.columns)
    use_features = st.sidebar.multiselect("Selecciona variables para AHP (Labor)", options=candidate_features_labor, default=candidate_features_labor[:3])

# ---------------------------
# Panel AHP: subir imagen explicativa y editar matriz
# ---------------------------
st.header("Panel AHP")
st.markdown("Sube una imagen explicativa (opcional) que se mostrará encima de la matriz de comparaciones. Luego edita la matriz o los pesos directamente.")

uploaded_img = st.file_uploader("Sube imagen explicativa (PNG/JPG)", type=['png','jpg','jpeg'])
if uploaded_img:
    st.image(uploaded_img, caption="Imagen explicativa AHP", use_column_width=True)

# Crear interfaz para editar matriz de comparaciones entre criterios (features)
st.subheader("Editar matriz de comparaciones (AHP)")
st.markdown("Usa valores 1,3,5,7,9 y recíprocos (1/3, 1/5...). Puedes editar cada par. Alternativa: editar pesos directos.")

# Preparar estructura de comparaciones para ahpy: dict of dicts
features = use_features.copy()
if len(features) < 2:
    st.warning("Selecciona al menos 2 features para ejecutar AHP.")
    st.stop()

# Construir dataframe triangular para edición
pairs = []
for i in range(len(features)):
    for j in range(i+1, len(features)):
        pairs.append({'feature_i': features[i], 'feature_j': features[j], 'value': 1.0})

pairs_df = pd.DataFrame(pairs)
# Mostrar editor del dataframe
edited_pairs = st.experimental_data_editor(pairs_df, num_rows="dynamic", key="ahp_pairs_editor")

# Convertir a comparisons dict para ahpy
comparisons = {}
for _, row in edited_pairs.iterrows():
    i = row['feature_i']
    j = row['feature_j']
    val = float(row['value']) if pd.notna(row['value']) else 1.0
    comparisons.setdefault(i, {})[j] = val

# Opción alternativa: editar pesos directos
st.markdown("**Opción:** editar pesos directos (sobreescribe la matriz si se usan).")
weights_manual = {}
use_manual_weights = st.checkbox("Usar pesos manuales (si activo, la matriz se ignorará para pesos)")
if use_manual_weights:
    col1, col2 = st.columns(2)
    for f in features:
        weights_manual[f] = col1.number_input(f"Peso {f}", min_value=0.0, value=1.0, step=0.1, key=f"w_{f}")
    # Normalizar pesos
    total_w = sum(weights_manual.values())
    if total_w == 0:
        # evitar dividir por cero
        for k in weights_manual:
            weights_manual[k] = 1.0/len(weights_manual)
    else:
        for k in weights_manual:
            weights_manual[k] = weights_manual[k]/total_w

# ---------------------------
# Botón ejecutar
# ---------------------------
if st.button("Ejecutar clasificación y métricas"):
    # Elegir dataset
    if view_mode == 'Shipping':
        df_use = df_ship.copy()
        sku_col = sku_col_ship
        aggregations = {'Qty Shipped': qty_col_ship, 'Weight [Kg]': weight_col_ship, 'Boxes': boxes_col_ship}
    else:
        df_use = df_labor.copy()
        sku_col = sku_col_labor
        # Asumir que user seleccionó features pertinentes; si no existen, compute_summary rellenará con ceros
        aggregations = {'Qty Shipped': use_features[0] if len(use_features)>0 else use_features[0], 'Weight [Kg]': use_features[1] if len(use_features)>1 else use_features[0], 'Boxes': use_features[2] if len(use_features)>2 else use_features[0]}

    # Calcular summary por SKU
    summary = compute_summary(df_use, sku_col=sku_col, aggregations=aggregations)

    # ABC clásico
    abc_df = compute_abc(summary, qty_col='Qty Shipped', cuts=cuts)

    # AHP: construir comparisons dict (si usan pesos manuales, creamos comparisons que reflejen esos pesos)
    if use_manual_weights:
        # Si el usuario quiere usar pesos manuales, construiremos un faux-comparisons dict que refleje ratios de pesos
        comp_manual = {}
        for i in range(len(features)):
            for j in range(i+1, len(features)):
                a = features[i]; b = features[j]
                val = weights_manual[a] / weights_manual[b] if weights_manual[b] != 0 else 1.0
                comp_manual.setdefault(a, {})[b] = float(val)
        comparisons_to_use = comp_manual
    else:
        comparisons_to_use = comparisons

    # Ejecutar AHP y obtener summary ahp
    ahp_summary, criteria = compute_ahp(summary, features=features, comparisons_dict=comparisons_to_use, cuts=cuts)

    # Unir ABC clásico y AHP en un solo df comparativo (usar SKU como key)
    merged = pd.merge(abc_df[['SKU','Qty Shipped','Weight [Kg]','Boxes','cum%','ABC_class']], 
                      ahp_summary[['SKU','AHP_score','cum_AHP%','AHP_class'] + [f'{f}_norm' for f in features]],
                      on='SKU', how='outer').fillna(0)

    st.header("Resultados comparativos")
    st.subheader("Tabla de resumen (por SKU)")
    st.dataframe(merged)

    # Mostrar conteos por clase
    colA, colB, colC = st.columns(3)
    with colA:
        st.metric("ABC - A count", int((merged['ABC_class']=='A').sum()))
        st.metric("AHP - A count", int((merged['AHP_class']=='A').sum()))
    with colB:
        st.metric("ABC - B count", int((merged['ABC_class']=='B').sum()))
        st.metric("AHP - B count", int((merged['AHP_class']=='B').sum()))
    with colC:
        st.metric("ABC - C count", int((merged['ABC_class']=='C').sum()))
        st.metric("AHP - C count", int((merged['AHP_class']=='C').sum()))

    # Mostrar reporte ahpy si estuvo bien construido
    if criteria is not None:
        st.subheader("Reporte AHP (ahpy)")
        try:
            st.text(criteria.report())
        except Exception:
            st.write("No se pudo renderizar report() de ahpy; mostrando pesos y consistency_ratio si están disponibles.")
            st.write("Pesos (target_weights):", criteria.target_weights)
            try:
                st.write("Consistency ratio:", criteria.consistency_ratio)
            except Exception:
                st.write("Consistency ratio no disponible.")

    else:
        st.warning("No se generó un objeto 'criteria' válido. Se usaron pesos iguales o manuales.")

    # Métricas de similitud
    st.subheader("Métricas de similitud por clase y globales")
    # Para métricas, usar columnas numéricas escogidas (features)
    merged_for_metrics = merged.copy()
    # Asegurarse de tener AHP_score y las features normalizadas
    metrics_df, global_indices = compute_similarity_metrics(merged_for_metrics, features=[f'{f}_norm' for f in features], class_col='AHP_class')
    st.dataframe(metrics_df)
    st.write("Índices globales:", global_indices)

    # Boxplots comparativos: Qty Shipped / Weight [Kg] / Boxes vs classes (AHP_class y ABC_class)
    st.subheader("Boxplots comparativos")
    fig_col1, fig_col2 = st.columns(2)
    with fig_col1:
        st.markdown("**Qty Shipped por AHP_class (Plotly interactivo)**")
        try:
            fig_px = px.box(merged, x='AHP_class', y='Qty Shipped', points="all", title="Qty Shipped vs AHP_class")
            st.plotly_chart(fig_px, use_container_width=True)
        except Exception as e:
            st.write("No se pudo generar Plotly:", e)
    with fig_col2:
        st.markdown("**Qty Shipped por ABC_class (Plotly interactivo)**")
        try:
            fig_px2 = px.box(merged, x='ABC_class', y='Qty Shipped', points="all", title="Qty Shipped vs ABC_class")
            st.plotly_chart(fig_px2, use_container_width=True)
        except Exception as e:
            st.write("No se pudo generar Plotly:", e)

    # Generar también gráficos estáticos con matplotlib/seaborn (por compatibilidad con el ejemplo)
    st.markdown("**Gráficos estáticos (matplotlib/seaborn)**")
    fig, axes = plt.subplots(1,3, figsize=(18,5))
    sns.boxplot(data=merged, x='AHP_class', y='Qty Shipped', ax=axes[0])
    axes[0].set_title("Qty Shipped vs AHP_class")
    sns.boxplot(data=merged, x='AHP_class', y='Weight [Kg]', ax=axes[1])
    axes[1].set_title("Weight [Kg] vs AHP_class")
    sns.boxplot(data=merged, x='AHP_class', y='Boxes', ax=axes[2])
    axes[2].set_title("Boxes vs AHP_class")
    st.pyplot(fig)

    # Botón de descarga
    st.subheader("Descargar resumen")
    to_download = merged.copy()
    csv = to_download.to_csv(index=False).encode('utf-8')
    st.download_button(label="Descargar CSV", data=csv, file_name=f"{view_mode}_summary.csv", mime='text/csv')

    # Mensajes de error/advertencia
    # Consistencia AHP
    try:
        if criteria is not None:
            cr = getattr(criteria, 'consistency_ratio', None)
            if cr is not None:
                if cr > 0.1:
                    st.warning(f"Consistency ratio AHP = {cr:.3f} > 0.1. Revisa la matriz de comparaciones; puede no ser consistente.")
                else:
                    st.success(f"Consistency ratio AHP = {cr:.3f}. Consistencia aceptable.")
    except Exception:
        st.info("No fue posible obtener la consistency_ratio del objeto ahpy.")

    st.success("Ejecución completada.")

# Fin del botón ejecutar
